### OpenCV: Open Computer Vision Library


In [8]:
import cv2
import numpy as np

from objloader_simple import OBJ

fox = OBJ("models/fox.obj")

In [14]:
frame = np.ones([480, 680,3], dtype=np.uint8)
screen_height, screen_width, z = frame.shape

f_near = 0.1
f_far = 1000
f_fov = 90
f_aspect_ratio = screen_height / screen_width
f_fov_rad = 1 / np.tan(f_fov * 0.5 /180 * np.pi)

af = f_aspect_ratio * f_fov_rad
f = f_fov_rad
q = f_far/(f_far-f_near)

mat_proj =  np.matrix([[ af, 0,          0, 0], 
                       [  0, f,          0, 0], 
                       [  0, 0,          q, 1],
                       [  0, 0,  -f_near*q, 0]])

def persp_transf(face):
    face = np.append(face, np.ones([3,1]), axis=1)
    res = []
    for line in face:
        l = np.matmul(line, mat_proj)
        w = l.item(3)
        if(w !=0):
            l = l * 1/w
        
        l=l[:,:3]
        res.append(l)
    return np.squeeze(np.asarray(res))

In [34]:
def render(img, obj, x_angle, y_angle, z_angle, wires):
    vertices = obj.vertices
    
    # rotation Matrix
    rotation_x = np.matrix(
        [
            [1,               0,                0], 
            [0, np.cos(x_angle), -np.sin(x_angle)], 
            [0, np.sin(x_angle),  np.cos(x_angle)]
        ]
    )

    rotation_y = np.matrix(
        [
            [np.cos(y_angle),  0, np.sin(y_angle)], 
            [              0,  1,               0], 
            [-np.sin(y_angle), 0, np.cos(y_angle)]
        ]
    )

    rotation_z = np.matrix(
        [
            [np.cos(z_angle), -np.sin(z_angle), 0],
            [np.sin(z_angle),  np.cos(z_angle), 0],
            [0              ,                0, 1]
        ])
    
    sorted_faces = []
    for face in obj.faces:
        face_vertices = face[0]
        points = np.array([vertices[vertex - 1] for vertex in face_vertices])
        
        object_m, object_n = points.shape
        
        # Rotate With
        
        points = [np.matmul(row,rotation_x) for row in points]
        points = [np.matmul(row,rotation_y) for row in points]
        points = [np.matmul(row,rotation_z) for row in points]
        points = np.squeeze(np.asarray(points))
        
        #Move from z
        x_y_move_matrix = np.ones([object_m,2], dtype=np.uint8)*1
        z_move_matrix = np.ones([object_m,1])*5
        z_move_matrix = np.append(x_y_move_matrix, z_move_matrix, axis=1)
        points = np.add(points,z_move_matrix)
        
        
        # Get Normals and check if is visible
        line1 = points[1] - points[0]
        line2 = points[2] - points[0]
        
        normal = np.cross(line1,line2)
        norm_len = np.linalg.norm(normal)
        normal = (normal*1/norm_len)
        
        if np.dot(normal,points[0]) < 0:
             #Lighting!
            light_len = np.linalg.norm(directional_light)
            light_normal = (directional_light*1/light_len)
            
            lum_val = np.dot(normal,light_normal)
            
            # Proyecting Points
            points = persp_transf(points)

            # move to somewhere on the screen with a translation matrix
            x_y_move_matrix = np.ones([object_m,2], dtype=np.uint8)*1.5
            z_move_matrix = np.zeros([object_m,1])
            move_matrix = np.append(x_y_move_matrix, z_move_matrix, axis=1)
            points = np.add(points,move_matrix)

            #Scale into view with scale matrix
            x_scale = 0.5 * screen_width
            y_scale = 0.5 * screen_height
            z_scale = 1
            scale_matrix = np.array([[x_scale, y_scale, z_scale],
                                     [x_scale, y_scale, z_scale],
                                     [x_scale, y_scale, z_scale]])
            points = points * scale_matrix
            
            p_light = np.array([np.append(f, lum_val) for f in points])
            sorted_faces.append(p_light)
            
    sorted_faces.sort(key= lambda point: (point[0][2] + point[1][2] + point[2][2])/3) 
    for points in sorted_faces:
            #Transform to 2d int array
            imgpts = np.int32(points[:,:2])
            
            # Add wires
            if wires:
                line_color =(0, 44, 150)
                line_color = list(line_color[::-1])
                line_color = [int(c) for c in line_color]

                cv2.polylines(img,[imgpts],True,line_color, thickness=2)
                
            #Fill triangle
            color = np.array([0, 204, 255]) * points[0][3]
            color = list(color[::-1])
            color = [int(c) for c in color]
                        
            cv2.fillConvexPoly(img, imgpts, color)
    return img

In [35]:
directional_light = np.array([0,0,-1])
x_angle = 0
y_angle = 0
z_angle = 3.12
wires = False
rendering = True
while rendering:
    frame = np.ones([720, 1280,3], dtype=np.uint8)
    frame = render(frame, fox, x_angle, y_angle, z_angle, wires)
    cv2.imshow("frame", frame)
    key = cv2.waitKey(33)
    
    if key == ord("w"):
        x_angle += 0.04
    if key == ord("s"):
        x_angle -= 0.04
    
    if key == ord("a"):
        y_angle -= 0.04
    if key == ord("d"):
        y_angle += 0.04
        
    if key == ord("q"):
        z_angle -= 0.04
    if key == ord("e"):
        z_angle += 0.04
        
    if key == ord("x"):
        wires = not wires
    
    if key == ord("p"):
        # remember remember to kill the frame
        cv2.destroyAllWindows()
        rendering = False